# Scope
scope referes to namespace  
  
namespace: Where a name lives. When a name is used, Python creates, changes or looks up the name in namespace.

- If a variable is assigned inside a **def**, it's **local** to that function.
- If a variable is assigned in an enclosing **def**, it' **nonlocal ** to nested functions.
- If a variable is assigned outside all **def**s, it's **gloabl** to the entire file

- Any assignment without **global** and **nonlocal** in a function is considered as **local**s while in-place changes to mutable objects is not.
- Each call to a functino creates a new local scope.

In [40]:
# Assignment
L = [1, 2, 3]

def change(L):
    L = [1, 2, 3, 4]
    
change(L)
print(L)    # No Change

[1, 2, 3]


In [39]:
# in-place change
L = [1, 2, 3]

def change(L):
    L.append(4)
    
change(L)
print(L)

[1, 2, 3, 4]


### Looking Up a Name: LEGB rule
- Looking Up Rule:
    - Local → Enclosing Function Locals → Global → Built-in

### Comprehension vairables
In Python3, vaiables in all comprehension forms(e.g. generator, list, dict, set) are considered as locals.  
In Python2, list comprehensions map their names to the scope outside the expression.  

```
By contrast, for loop never localize their variables to the statement block in any python
```

## Reference vs Assignment
- reference (X)
    - looks for name using LEGB rule
- assignment (X = value)
    - Create or change the name in local by default.
    - If the name is declared **global**, the operations start from global scope
    - If the name is declared **nonlocal**(Python3 only), the operations look up only in enclosing functions

---

## Global Variables
- Assigned at the top level of file
- Must be declared only if they are assigned with a function
- May be referenced with a function without being declared

## Global vs Nonlocal
- **global**
    - Makes scope lookup begin in module scope and then built-in scope
    - Allows name to be assign
        - If not exist, it create name in module level
- **nonlocal**
    - Restricts scope lookup to just enclosing defs. No global or built-in lookups.
    - Requires that the names already exist

In [41]:
# Global example

def test():
    global a
    a = 5
    
test()
print(a)

5


In [42]:
# Nonlocal example

def tester(start):
    state = start           # state must exist before nonlocal is declared
    def nested(label):
        nonlocal state
        print(label, state)
        state += 1
    return nested

F = tester(0)
F('zero')
F('one')
F('two')

zero 0
one 1
two 2


## Function Attribute
Since Python2 does not support **nonlocal**, this could be a portable way to both versions.

In [35]:
def tester(start):
    def nested(label):
        print(label, nested.state)
        nested.state += 1
    nested.state = start
    return nested

F = tester(0)
F('zero')
F('one')
F('two')

zero 0
one 1
two 2


---

## Program Design Issue
### Minimize Global Variables
- It might be useful when using multi-thread, it can be used to shared memory between threads.  
    - Threads is commonly used for long-running tasks in GUIs

### Minimize Cross-File Changes
- It's possiable, but strongly not recommended.
- It's also possible to refine a built-in name. But this is often a bug and won't trigger any warning message. Also not recommended.

---

## Factory Functions
- Useful when creating event handler.

In [46]:
# This technique is often used with lambda

def maker(n):
    return lambda x: x**n

f = maker(3)
f(4)

64

### Loop vaiables may require defaults, not scopes

If a **lambda** or **def** is nested inside a loop, and it references an enclosing variable that is changed by that loop, all functions generated with the loop will have the same value - the value the referenced variable had in the last loop.

In [48]:
# The same value which is not expected

def makeActions():
    acts = []
    for i in range(5):
        acts.append(lambda x: i**x)
    return acts

acts = makeActions()

print(acts[0](2))         # 4**2, which is not expected
print(acts[1](2))
print(acts[2](2))

16
16
16


The enclosgin scope variable is looked up when the nested functions are latter called.  
Thus, the **i** won't change in the above example.

In [50]:
# Generating different functions which is as expected

def makeActions():
    acts = []
    for i in range(5):
        acts.append(lambda x, i=i: i**x)
    return acts

acts = makeActions()

print(acts[0](2))        # 0**2
print(acts[1](2))
print(acts[2](2))

0
1
4


The defaults are evaluated when the nested functions is created.